### Importing all Libraries

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sns.set(font_scale=1.5)
sns.set_style("whitegrid")
import sqlite3

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kaushik/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Keys required to connect to Twitter API

In [3]:
Consumer_Key="7ULNGRHNK9zJIxjc5KLxuMTeZ"
Consumer_Secret="1jPRtvBFxvZwUzd9Gk9gUvYSMnLHrBEQrcVieQjGeEDqSPnnTf"
Access_Token="1458418334081425410-GiOCzDRZDkCVCOxPgVcpBSv7454OQV"
Access_Secret="k58HmCgCHH7cfjChhMJyhJSHeZ0UGo6f0MjYNxR0H1gt6"

### Function to authenticate with Twitter API

In [4]:
def tweet_api_authentication(Consumer_Key, Consumer_Secret, Access_Token, Access_Secret):
    """
        /********------------------Function to authenticate with Twitter API------------------------------********/
             
                 - Consumer_Key: Provided by Twitter API dashboard
                 - Consumer_Secret: Secret key provided by Twitter API dashboard
                 - Access_Token: Token provided by Twitter API dashboard
                 - Access_Secret: Secret Access key provided by Twitter API dashboard
            
            Please check the twitter API docs for more information: https://developer.twitter.com/en/docs/twitter-api/getting-started/getting-access-to-the-twitter-api
        /********-----------------------------------------------------------------------------------------********/
    """
    auth = tw.OAuthHandler(Consumer_Key,Consumer_Secret)
    auth.set_access_token(Access_Token,Access_Secret)
    api = tw.API(auth, wait_on_rate_limit = True)
    return api
api = tweet_api_authentication(Consumer_Key, Consumer_Secret, Access_Token, Access_Secret)

### Function to fetch tweets for API

In [5]:
def fetch_tweets_from_api(key_word_string, api_object):
    """
        /*****------------------Function to fetch tweets from Twitter API-------------------------******/
        
            - key_word_string: Enter a string with keywords to fetch tweets
                               example: Food OR food OR Foodie OR Delicious OR Breakfast OR Lunch OR Dinner
            - api_object: Twitter API authenticated object. (Check func tweet_api_authentication() to create object)

        /******----------------------------------------------------------------------------------******/
    """
    search_tweets = key_word_string +  "-filter:retweets"
    tweets = tw.Cursor(api_object.search_tweets,
                            q = search_tweets,
                            lang="en",
                            tweet_mode="extended"
                            ).items(5)
    tweets_list = [[tweet.created_at, tweet.place, tweet.user.name, 
                         tweet.full_text] for tweet in tweets]
    
    return tweets_list

### Class to create a database and new tables for each tweet fetch

In [7]:
class database_operations():
    """
    
        /*****________________Class to write and read tweets into database________________*****/   
        
    """
    def __init__(self, table_name, tweets_list):
        """
        
            While initializing class object pass arguements:
                - table_name: Name of table to be created for tweets
                - tweets_list: List of tweets fetched from API (The list must contain, Date, Place, Username, Tweet)
                               Check func fetch_tweets_from_api() for more help
        """
        self.table_name = table_name
        self.tweets_list = tweets_list
    
    def get_table(self):
        """
        
            Returns the table name of current object
        """
        return self.table_name
    
    def create_connection(self):
        """
        
            Create Connection with tweets database using SQLLite3
        """
        conn = sqlite3.connect('tweets_database')
        cursor = conn.cursor()
        return [conn, cursor]
    
    def clean_tweet(self, txt):
        """
            Function to remove urls, symbols, retweet symbols
                - txt: Text to be cleaned
        """
        txt = re.sub(r'@[A-Za-z0-9_]+', '', txt) # Keeps text A-Z, a-z, 0-9
        txt = re.sub(r'#[A-Z0-9]+', '', txt)
        txt = re.sub(r'RT : ', '', txt)
        txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt) # remove url
        txt = re.sub(r'&amp;', '', txt)
        txt = re.sub(r'ðŸ™', '', txt)
        txt = re.sub(r'\n', ' ', txt) 
        return txt
        
    def create_table_from_dataframe(self):
        """
            Creating SQLLite3 Table from a pandas dataframe with given list of tweets
            Database Columns = ['Created at', 'Place', 'User', 'Text']
        """
        conn, cursor = self.create_connection()
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {self.table_name} (created_at text, place text, user text, tweet text, cleaned_tweets text)")
        conn.commit()
        tweets_df = pd.DataFrame(self.tweets_list, columns = ['Created at', 'Place', 'User', 'Tweets'])
        tweets_df['Cleaned_Tweets'] = tweets_df['Tweets'].apply(self.clean_tweet)
        tweets_df.to_sql(self.table_name, conn, if_exists='replace', index = False)
        
    def read_from_table(self):
        """
            Reading data from SQLLite3 DATABASE tweets_database 
            
        """
        conn, cursor = self.create_connection()
        cursor.execute(f"SELECT * FROM {self.table_name}")
        tweets_df = pd.DataFrame(cursor.fetchall(), columns = ['Created at', 'Place', 'User', 
                                                               'Tweets', 'Cleaned_Tweets'])
        return tweets_df

    # Food OR food OR Foodie OR Delicious OR Breakfast OR Lunch OR Dinner
key_words = "trump sucks"
tweets_list = fetch_tweets_from_api(key_words, api)
help(database_operations)
db = database_operations('food', tweets_list)
db.create_table_from_dataframe()
print(db.get_table())
tweets_df = db.read_from_table()
print("read \n", tweets_df.head())

Help on class database_operations in module __main__:

class database_operations(builtins.object)
 |  database_operations(table_name, tweets_list)
 |  
 |  /*****________________Class to write and read tweets into database________________*****/
 |  
 |  Methods defined here:
 |  
 |  __init__(self, table_name, tweets_list)
 |      While initializing class object pass arguements:
 |          - table_name: Name of table to be created for tweets
 |          - tweets_list: List of tweets fetched from API (The list must contain, Date, Place, Username, Tweet)
 |                         Check func fetch_tweets_from_api() for more help
 |  
 |  clean_tweet(self, txt)
 |      Function to remove urls, symbols, retweet symbols
 |          - txt: Text to be cleaned
 |  
 |  create_connection(self)
 |      Create Connection with tweets database using SQLLite3
 |  
 |  create_table_from_dataframe(self)
 |      Creating SQLLite3 Table from a pandas dataframe with given list of tweets
 |      Database

In [14]:
class handle_model():
    def __init__(self, ml_model):
        """
            While initializing class you need to pass the model as an arguement
                - model: The model you want to use for Sentiment Analysis
        """
        self.ml_model = ml_model
    
    def predict(self, tweets_df):
        """
            Function to predict sentiments of given tweets
                - tweets_df: tweets dataframe created from fetching tweets from API, more help look at
                  class database_operations
        """
        key_list = ['neg', 'neu', 'pos']
        for index, tweet in tweets_df['Cleaned_Tweets'].iteritems():
            score = self.ml_model.polarity_scores(tweet)
            print(f"tweet: {tweet}")
            print(f"this tweet is {round(score['neg']*100, 2)}% negative, {round(score['neu']*100, 2)}% neutral, {round(score['pos']*100, 2)}% positive")
            print()

m = handle_model(SentimentIntensityAnalyzer())            
m.predict(tweets_df)

tweet: What sucks is that Trump can literally confess to felonies on TV and not lose any supporters. 
this tweet is 35.5% negative, 54.9% neutral, 9.5% positive

tweet:  Yeah, Garland still sucks. Let me know when he indicts Trump, till then, everything else is fixing the car’s flat tire while the engine is on fire.
this tweet is 15.8% negative, 77.2% neutral, 7.1% positive

tweet: GOP heroes like Trump, Flynn, Boebert, MTG, Gaetz, Cawthorne, Cruz, Paul, Gomert, Jordan are the kinds of people good people want nothing to do with.  The GOP fights science, sensible gun control, promote racial/ethnic hatred, hate mask  vax mandates... In short - the GOP sucks.
this tweet is 25.0% negative, 54.8% neutral, 20.3% positive

tweet:  It sucks, but also I’m interested in seeing where that trend goes in the next couple years. Like trump’s spike in his final year was absolutely massive. If Biden also keeps screwing this up, he’s basically handing trump a second term so that blows.
this tweet is 8.2

In [19]:
from flask import Flask, request, redirect, url_for, session
from werkzeug.utils import secure_filename
import logging

In [20]:
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('HELLO WORLD')

In [61]:
app = Flask(__name__)

In [62]:
@app.route('/upload',methods=['POST'])
def uploadFile():
    app.logger.info("The name is " + request.form['Name'])
    return 'Response'

In [63]:
app.run(debug=True, host="127.0.0.1", use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:__main__:The name is Kaushik
INFO:werkzeug:127.0.0.1 - - [06/Dec/2021 22:48:15] "POST /upload HTTP/1.1" 200 -
